# HAI Deepfake - Inference (prithivMLmods Model)

**모델**: `prithivMLmods/Deep-Fake-Detector-v2-Model` (Hugging Face)
**전략**: Top-K Average, RetinaFace Crop Data

In [ ]:
!nvidia-smi
from google.colab import drive
drive.mount('/content/drive')

!pip install -q transformers torch pillow

In [ ]:
import os

# 데이터 준비
DRIVE_ZIP_PATH = '/content/drive/MyDrive/HAI_Deepfake/processed_test_data.zip'
LOCAL_DATA_DIR = '/content/processed_test_data'

if not os.path.exists(LOCAL_DATA_DIR):
    print("📂 데이터 복사 및 압축 해제 중...")
    os.makedirs(LOCAL_DATA_DIR, exist_ok=True)
    !cp {DRIVE_ZIP_PATH} /content/data.zip
    !unzip -q /content/data.zip -d {LOCAL_DATA_DIR}
    print("✅ 데이터 준비 완료!")
else:
    print("✅ 데이터가 이미 존재합니다.")

In [ ]:
import torch
import pandas as pd
import numpy as np
from pathlib import Path
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
import torch.nn.functional as F
from transformers import AutoModelForImageClassification, AutoImageProcessor

# 설정
MODEL_NAME = "prithivMLmods/Deep-Fake-Detector-v2-Model"
OUTPUT_CSV = "submission_prithiv.csv"

class ProcessedDataset(Dataset):
    def __init__(self, data_dir, processor=None):
        self.data_dir = Path(data_dir)
        self.processor = processor
        self.items = sorted([item for item in self.data_dir.glob('*') if not item.name.startswith('.')])
        
        if len(self.items) == 1 and self.items[0].is_dir():
            self.data_dir = self.items[0]
            self.items = sorted([item for item in self.data_dir.glob('*') if not item.name.startswith('.')])

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        item_path = self.items[idx]
        pixel_values_list = []
        
        # 이미지 로드
        if item_path.is_dir():
            img_paths = sorted(list(item_path.glob('*.jpg')))
            frames = [Image.open(p).convert("RGB") for p in img_paths]
        else:
            frames = [Image.open(item_path).convert("RGB")]

        # Processor 적용 (Hugging Face)
        if frames:
            inputs = self.processor(images=frames, return_tensors="pt")
            pixel_values_list = inputs['pixel_values']
        else:
            return torch.zeros(1, 3, 224, 224), item_path.name

        return pixel_values_list, item_path.name

def run_inference():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    print(f"Loading model: {MODEL_NAME}")
    model = AutoModelForImageClassification.from_pretrained(MODEL_NAME).to(device)
    processor = AutoImageProcessor.from_pretrained(MODEL_NAME)
    model.eval()
    
    # 라벨 확인
    print(f"Label Map: {model.config.id2label}")

    dataset = ProcessedDataset(LOCAL_DATA_DIR, processor=processor)
    dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=2)

    results = {}
    print(f"🚀 Starting inference...")
    
    with torch.no_grad():
        for pixel_values, filenames in tqdm(dataloader):
            # (1, T, C, H, W) -> (T, C, H, W)
            pixel_values = pixel_values.squeeze(0).to(device)
            filename = filenames[0]
            
            outputs = model(pixel_values)
            probs = F.softmax(outputs.logits, dim=1)
            
            # Fake 확률 (보통 Label 1)
            fake_probs = probs[:, 1]
            
            # Top-K Average (상위 5개)
            k = min(5, len(fake_probs))
            top_k_probs, _ = torch.topk(fake_probs, k)
            final_prob = float(top_k_probs.mean().cpu().item())
            
            clean_filename = filename.replace('_frames', '')
            results[clean_filename] = final_prob

    return results

results = run_inference()

submission = pd.DataFrame(list(results.items()), columns=['filename', 'prob'])
submission.to_csv(OUTPUT_CSV, index=False)
print(f"✅ Submission saved: {OUTPUT_CSV}")

BACKUP_PATH = '/content/drive/MyDrive/HAI_Deepfake/submission_prithiv.csv'
!cp {OUTPUT_CSV} {BACKUP_PATH}
print(f"💾 Drive Backup: {BACKUP_PATH}")